In [1]:
from pymongo import MongoClient
import pandas as pd
import certifi
import json
import math
from datetime import datetime
import time
from IPython.display import clear_output
import configparser

In [2]:
# Load configuration from file
config = configparser.ConfigParser()
config.read('config.cfg')

# Access the MongoDB URI
MONGO_URI = config.get('mongo', 'MONGO_URI')
DATABASE_NAME = config.get('mongo', 'DATABASE_NAME')
COLLECTION_NAME = config.get('mongo', 'COLLECTION_NAME')
COLLECTION_NAME1 = config.get('mongo', 'COLLECTION_NAME1')

In [4]:
# Connect to the MongoDB server
client = MongoClient(MONGO_URI, tlsCAFile=certifi.where())

# Select the database and collection
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]
collection1 = db[COLLECTION_NAME1]
# Fetch the data
data = list(collection.find())

In [10]:
# Function to read flight data from a JSON file
def read_flight_data(data):
    return json.load(data)

# Function to parse the flight data
def parse_data(flight_data):
    parsed_data = []
    for entry in flight_data:
        flight_id = entry['flight_id']
        timestamp = datetime.fromisoformat(entry['ts'])
        lat = entry['location']['lat']
        lon = entry['location']['long']
        speed = entry['velocity']['speed']
        heading = entry['velocity']['heading']
        distance_db = entry['distance_to_arrival']
        delay_length = entry['extra_length']

        parsed_data.append({
            'FlightID': flight_id,
            'Timestamp': timestamp,
            'Latitude': lat,
            'Longitude': lon,
            'Speed': speed,
            'Heading': heading,
            'Distance': distance_db,
            'Delay_length': delay_length
        })
    return parsed_data

# Function to calculate the time left to reach the destination
def time_left(distance, speed):
    if speed > 0:
        return distance / speed
    else:
        return float('inf')  # Infinite time if speed is zero

# A function to calculate cost
def calculate_cost(speed, distance, delay_cost_per_minute, delay_time):
    time_left_in_hours = time_left(distance, speed)
    fuel_cost = 1500 * time_left_in_hours #average price per hour of fuel is 1500$
    delay_cost = delay_cost_per_minute * delay_time * 60
    total_cost = fuel_cost + delay_cost
    return total_cost

# Function to process the data and calculate costs iteratively
def process_flight_data(delay_cost_per_minute):
    flight_data = data
    parsed_data = parse_data(flight_data)

    prev_fcr = None
    for i, entry in enumerate(parsed_data):
        if i == 0:
            continue

        prev_entry = parsed_data[i - 1]
        current_entry = entry

        #distance_traveled = haversine(prev_entry['Latitude'], prev_entry['Longitude'], current_entry['Latitude'], current_entry['Longitude'])
        distance_to_destination = current_entry['Distance']
        speed = current_entry['Speed']
        prev_speed = prev_entry['Speed']
        estimated_time_left = time_left(distance_to_destination, speed)

        # Calculate total cost for the current data point
        delay_time = time_left(current_entry['Delay_length'], speed)
        total_cost = calculate_cost(speed, distance_to_destination, delay_cost_per_minute, delay_time)
        delay_cost = delay_time * delay_cost_per_minute * 60 # by 60 because delay_time is given in hours
        fuel_cost = 1500 * estimated_time_left
        # Clear the previous output
        clear_output(wait=True)

        # Create a document to insert into MongoDB
        document = {
            "Timestamp": current_entry['Timestamp'],
            "DistanceToDestination": distance_to_destination,
            "EstimatedTimeLeft": estimated_time_left,
            "DelayTime": delay_time,
            "DelayCost": delay_cost,
            "FuelCostPerHour": fuel_cost,
            "TotalCostPerHour": total_cost
        }

        # Insert the document into the collection
        collection1.insert_one(document)

        # Display the results
        print(f"Timestamp: {current_entry['Timestamp']}")
        #print(f"Distance Traveled: {distance_traveled:.2f} km")
        print(f"Distance to Destination: {distance_to_destination:.2f} km")
        print(f"Estimated Time Left: {estimated_time_left:.2f} hours")
        print(f"Delay Time: {delay_time:.2f} hours")
        print(f"Delay Cost: {delay_cost:.2f} EUR")
        print(f"Fuel Cost: {fuel_cost:.2f} EUR")
        print(f"Total Cost: {total_cost:.2f} EUR")
        print("-----")

        # Simulate real-time delay (e.g., 1 second)
        time.sleep(1)

# Example parameters
delay_cost_per_minute = 105  # Example delay cost per minute

# Process the flight data and calculate costs iteratively
process_flight_data(delay_cost_per_minute)




Timestamp: 2024-07-16 16:16:04.606484
Distance to Destination: 205.42 km
Estimated Time Left: 0.85 hours
Delay Time: 0.00 hours
Delay Cost: 0.00 EUR
Fuel Cost: 1280.10 EUR
Total Cost: 1280.10 EUR
-----


KeyboardInterrupt: 